In [227]:
import os
import pandas as pd
import re
from PyPDF2 import PdfReader

    

In [266]:
file = '../Downloaded Data/2021/Abu Dhabi/2021_22_uae_f1_p1_timing_firstpracticesessionlaptimes_v01.pdf'
file = '../Downloaded Data/2021/British/2021_10_gbr_f1_p1_timing_firstpracticesessionlaptimes_v01.pdf'

In [285]:
file_list = []
for path, subdirs, files in os.walk('../Downloaded Data/'):
    for name in files:
        file_list.append(os.path.join(path, name))
file_list = [file for file in file_list if "firstpracticesessionlaptimes" in file]

times = pd.DataFrame()

for file in file_list:
    
    race_info = re.split('-',re.sub('.+Data/(\d{4})/([^/]+)/.+',r'\1-\2', file))
    race_year = race_info[0]
    race_name = race_info[1]
    session_name = "FP1"

    reader = PdfReader(file)
    text = ""
    for page in reader.pages:
        text += page.extract_text() + "\n"
        
    text = text.replace("\n","")
    text = re.sub('(\.\d{3})(\d+)',r'\1 \2', text)
    text = re.sub('(\d+)(\s{2,})',r'-\1 ', text)
    text = re.split('-', text)
    text = [data for data in text if data]

    for data in text:
        data = re.sub("(\.\d{3})[a-zA-Z].+",r"\1",data)
        data = re.sub("(\d)(P)(\d)",r"\1 \2 \3", data)
        data = re.sub("NOTIME", " ", data)
        data = re.sub(" (\d{1,2} (P )?\d:\d{2}\.\d{3})",r"-\1", data)
        data = re.split("-", data)
        if len(data) > 1:
            driver_info = re.split(' ', data[0], maxsplit = 1)
            driver_name = driver_info[1]
            driver_number = driver_info[0]
            driver_laptimes = data[1:]
            for lap in driver_laptimes:
                lap_pit = "P" if "P" in lap else ""
                lap = re.sub("P ","", lap)
                lap_info = re.split(' ', lap, maxsplit = 1)
                lap_number = int(lap_info[0])
                lap_time = lap_info[1]
                temp_df = pd.DataFrame({"Race Year":[race_year], "Race Name":[race_name], "Session Name":[session_name], "Driver Name":[driver_name], "Driver Number":[driver_number], "Lap Number":[lap_number], "Lap Time":[lap_time], "Pit":[lap_pit]})
                times = times.append(temp_df)

In [293]:
times

,Race Year,Race Name,Session Name,Driver Name,Driver Number,Lap Number,Lap Time,Pit
0,2019,Belgian,FP1,RICCIARDO 1 P 11:04:022 20:55.949,3Daniel,3,1:46.790,
0,2019,Belgian,FP1,RICCIARDO 1 P 11:04:022 20:55.949,3Daniel,4,2:15.317,
0,2019,Belgian,FP1,RICCIARDO 1 P 11:04:022 20:55.949,3Daniel,5,2:21.768,
0,2019,Belgian,FP1,RICCIARDO 1 P 11:04:022 20:55.949,3Daniel,6,2:10.587,
0,2019,Belgian,FP1,RICCIARDO 1 P 11:04:022 20:55.949,3Daniel,7,1:47.051,
...,...,...,...,...,...,...,...,...
0,2020,Styrian,FP1,Robert KUBICA,88,26,1:07.504,
0,2020,Styrian,FP1,Robert KUBICA,88,27,1:34.911,
0,2020,Styrian,FP1,Robert KUBICA,88,28,1:07.856,
0,2020,Styrian,FP1,Robert KUBICA,88,29,1:55.969,
